In [1]:
import pdfplumber
from langchain.text_splitter import RecursiveCharacterTextSplitter


def extract_pdf_text(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text


# Paths to your PDFs
pdf1_path = "../data/FAS32.pdf"
pdf2_path = "../data/SS9.pdf"

# Extract text
ijarah32_text = extract_pdf_text(pdf1_path)
ss9_text = extract_pdf_text(pdf2_path)

# Combine for RAG
all_docs = [
    {"source": "FAS32.pdf", "text": ijarah32_text},
    {"source": "SS9-Ijarah.pdf", "text": ss9_text}
]

# Chunk the text for retrieval
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
rag_chunks = []
for doc in all_docs:
    for chunk in splitter.split_text(doc["text"]):
        rag_chunks.append({"source": doc["source"], "text": chunk})

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

In [3]:
from langchain_openai import OpenAIEmbeddings
import faiss
import numpy as np

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
chunk_texts = [chunk["text"] for chunk in rag_chunks]
chunk_vectors = embeddings.embed_documents(chunk_texts)

dimension = len(chunk_vectors[0])
index = faiss.IndexFlatL2(dimension)
index.add(np.array(chunk_vectors).astype("float32"))

In [4]:
def retrieve_relevant_chunks(query, top_k=3):
    query_vec = np.array(embeddings.embed_query(
        query)).astype("float32").reshape(1, -1)
    D, I = index.search(query_vec, top_k)
    results = []
    for idx in I[0]:
        results.append(rag_chunks[idx])
    return results


# Example: Retrieve relevant chunks for "Ijarah MBT Accounting"
relevant_chunks = retrieve_relevant_chunks(
    "Ijarah MBT Accounting in Lessee’s books")
for i, chunk in enumerate(relevant_chunks):
    print(
        f"--- Chunk {i+1} from {chunk['source']} ---\n{chunk['text'][:500]}\n")

--- Chunk 1 from Ijarah (32).pdf ---
Ijarah MBT through gradual transfer / Diminishing Musharaka Ijarah .......................................................... 38
Ijarah commencement date vs. inception date – issue of future Ijarah ........................................................ 38
Accounting and financial reporting by the Lessor .......................................................................................... 39

--- Chunk 2 from Ijarah (32).pdf ---
if the lessee incurs the same, these shall be recorded as recoverable from lessor (even if lessor has a
supplementary rental mechanism to recover the same, which shall be accounted for accordingly as a
revenue).
Ijarah MBT through gradual transfer / Diminishing Musharaka Ijarah
BC27 The Board after taking into account industry comments agreed that the accounting for Ijarah MBT
through gradual transfer under FAS 8 was having practical impediments and required improvement.

--- Chunk 3 from Ijarah (32).pdf ---
Ijarah MBT

In [5]:
import re


def calculate_ijarah_entries(params):
    prime_cost = params["purchase_price"] + \
        params["import_tax"] + params["freight"]
    rou = prime_cost - params["terminal_value"]
    total_rentals = params["yearly_rental"] * params["ijarah_term"]
    deferred_ijarah_cost = total_rentals - rou
    terminal_value_diff = params["residual_value"] - params["terminal_value"]
    amortizable_amount = rou - terminal_value_diff

    return {
        "prime_cost": prime_cost,
        "rou": rou,
        "total_rentals": total_rentals,
        "deferred_ijarah_cost": deferred_ijarah_cost,
        "terminal_value_diff": terminal_value_diff,
        "amortizable_amount": amortizable_amount
    }

In [7]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


def extract_numbers_from_scenario_llm(scenario, llm=None):
    if llm is None:
        llm = OpenAI(temperature=0)

    prompt = PromptTemplate(
        input_variables=["scenario"],
        template="""
You are an expert accountant. Extract the following numbers from the scenario below. If a value is missing, return 0.

Scenario:
{scenario}

Return a JSON object with these keys:
- purchase_price
- import_tax
- freight
- ijarah_term
- residual_value
- terminal_value
- yearly_rental

Example output:
{{
  "purchase_price": 450000,
  "import_tax": 12000,
  "freight": 30000,
  "ijarah_term": 2,
  "residual_value": 5000,
  "terminal_value": 3000,
  "yearly_rental": 300000
}}
"""
    )

    chain = LLMChain(llm=llm, prompt=prompt)
    response = chain.run(scenario=scenario)

    # Parse the JSON from the LLM output
    import json
    import re
    match = re.search(r"\{.*\}", response, re.DOTALL)
    if match:
        return json.loads(match.group(0))
    else:
        raise ValueError("Could not parse LLM output as JSON:\n" + response)

In [12]:
def ijarah_agent(scenario, query="Ijarah MBT Accounting in Lessee’s books", top_k=3):
    # 1. Retrieve relevant chunks
    relevant_chunks = retrieve_relevant_chunks(query, top_k=top_k)
    print("Relevant Chunks from PDFs:")
    for i, chunk in enumerate(relevant_chunks):
        print(
            f"--- Chunk {i+1} from {chunk['source']} ---\n{chunk['text'][:300]}...\n")

    # 2. Extract numbers from scenario
    params = extract_numbers_from_scenario_llm(scenario)
    print("Extracted Parameters:", params)

    # 3. Calculate entries
    results = calculate_ijarah_entries(params)
    print("\n--- Accounting Calculations ---")
    print(f"Prime cost: {results['prime_cost']}")
    print(f"ROU: {results['rou']}")
    print(f"Total rentals: {results['total_rentals']}")
    print(f"Deferred Ijarah Cost: {results['deferred_ijarah_cost']}")
    print(f"Terminal value diff: {results['terminal_value_diff']}")
    print(f"Amortizable Amount: {results['amortizable_amount']}")

    print("\n--- Journal Entry ---")
    print(f"Dr. Right of Use Asset (ROU) USD {results['rou']:,}")
    print(f"Dr. Deferred Ijarah Cost USD {results['deferred_ijarah_cost']:,}")
    print(f"    Cr. Ijarah Liability USD {results['total_rentals']:,}")

    print("\n--- Amortizable Amount Calculation ---")
    print(f"Cost of ROU: {results['rou']:,}")
    print(
        f"Less: Terminal value difference (Residual {params['residual_value']:,} − Purchase {params['terminal_value']:,}): {results['terminal_value_diff']:,}")
    print(f"Amortizable Amount: {results['amortizable_amount']:,}")

    return results, relevant_chunks


# Example usage:
scenario = """
The start of 2019, specifically January 1st, marked Alpha Islamic Bank's entry into an Ijarah Muntahia Bittamleek (MBT) agreement as the Lessee, partnering with Super Generators. The subject of this Ijarah was a heavy-duty generator, which Super Generators had purchased for USD 450,000. To this initial cost, Super Generators added USD 12,000 for import duties and a further USD 30,000 to cover freight expenses.
This Ijarah arrangement is set for a duration of two years. At the end of this period, the generator is projected to have a residual value of USD 5,000. Critically, it is anticipated with high certainty that Alpha Islamic Bank will exercise its option to acquire the asset by purchasing it for USD 3,000. Throughout the Ijarah term, Alpha Islamic Bank will be responsible for annual rental payments amounting to USD 300,000 and will also amortize the 'right-of-use' asset on a yearly basis.
"""

ijarah_agent(scenario)

Relevant Chunks from PDFs:
--- Chunk 1 from FAS32.pdf ---
Ijarah MBT through gradual transfer / Diminishing Musharaka Ijarah .......................................................... 38
Ijarah commencement date vs. inception date – issue of future Ijarah ........................................................ 38
Accounting and financial reporting by the ...

--- Chunk 2 from FAS32.pdf ---
if the lessee incurs the same, these shall be recorded as recoverable from lessor (even if lessor has a
supplementary rental mechanism to recover the same, which shall be accounted for accordingly as a
revenue).
Ijarah MBT through gradual transfer / Diminishing Musharaka Ijarah
BC27 The Board after ...

--- Chunk 3 from FAS32.pdf ---
Ijarah MBT through gradual transfer – special considerations ................................................................. 28
De-recognition in the books of the seller / transferor (previously lessor) ..................................................... 28
Ijarah M

({'prime_cost': 492000,
  'rou': 489000,
  'total_rentals': 600000,
  'deferred_ijarah_cost': 111000,
  'terminal_value_diff': 2000,
  'amortizable_amount': 487000},
 [{'source': 'FAS32.pdf',
   'text': 'Ijarah MBT through gradual transfer / Diminishing Musharaka Ijarah .......................................................... 38\nIjarah commencement date vs. inception date – issue of future Ijarah ........................................................ 38\nAccounting and financial reporting by the Lessor .......................................................................................... 39'},
  {'source': 'FAS32.pdf',
   'text': 'if the lessee incurs the same, these shall be recorded as recoverable from lessor (even if lessor has a\nsupplementary rental mechanism to recover the same, which shall be accounted for accordingly as a\nrevenue).\nIjarah MBT through gradual transfer / Diminishing Musharaka Ijarah\nBC27 The Board after taking into account industry comments agreed tha